In [1]:
import torch
from ogb.nodeproppred import PygNodePropPredDataset
from torch_geometric import loader

dataset = PygNodePropPredDataset(name = "ogbn-products", root = 'dataset/')
 
split_idx = dataset.get_idx_split()
data = dataset[0]

# Convert split indices to boolean masks and add them to `data`.
for key, idx in split_idx.items():
    mask = torch.zeros(data.num_nodes, dtype=torch.bool)
    mask[idx] = True
    data[f'{key}_mask'] = mask
    

In [2]:
from sklearn.metrics import pairwise_distances
import numpy as np
from scipy.linalg import eigh
import warnings
import networkx as nx

class LE:
    
    def __init__(self, X:np.ndarray, dim:int, k:int = 2, eps = None, graph:str = 'k-nearest', weights:str = 'simple', 
                 sigma:float = 0.1, laplacian:str = 'unnormalized', opt_eps_jumps:float = 1.5):
        """
        LE object
        Parameters
        ----------
        
        X: nxd matrix
        
        dim: number of coordinates
        
        k: number of neighbours. Only used if graph = 'k-nearest'
        
        eps: epsilon hyperparameter. Only used if graph = 'eps'. 
        If is set to None, then epsilon is computed to be the 
        minimum one which guarantees G to be connected
        
        graph: if set to 'k-nearest', two points are neighbours 
        if one is the k nearest point of the other. 
        If set to 'eps', two points are neighbours if their 
        distance is less than epsilon
        
        weights: if set to 'heat kernel', the similarity between 
        two points is computed using the heat kernel approach.
        If set to 'simple', the weight between two points is 1
        if they are connected and 0 otherwise. If set to 'rbf'
        the similarity between two points is computed using the 
        gaussian kernel approach.
        
        sigma: coefficient for gaussian kernel or heat kernel
        
        laplacian: if set to 'unnormalized', eigenvectors are 
        obtained by solving the generalized eigenvalue problem 
        Ly = λDy where L is the unnormalized laplacian matrix.
        If set to 'random', eigenvectors are obtained by decomposing
        the Random Walk Normalized Laplacian matrix. If set to 
        'symmetrized', eigenvectors are obtained by decomposing
        the Symmetrized Normalized Laplacian
        
        opt_eps_jumps: increasing factor for epsilon
        """
        
        self.X = X
        self.dim = dim
        self.k = k
        self.eps = eps
        if graph not in ['k-nearest', 'eps']:
            raise ValueError("graph is expected to be a graph name; 'eps' or 'k-nearest', got {} instead".format(graph))
        self.graph = graph
        if weights not in ['simple', 'heat kernel', 'rbf']:
            raise ValueError("weights is expected to be a weight name; 'simple' or 'heat kernel', got {} instead".format(weights))
        self.weights = weights
        self.sigma = sigma
        self.n = self.X.shape[0]
        if laplacian not in ['unnormalized', 'random', 'symmetrized']:
            raise ValueError("laplacian is expected to be a laplacian name; 'unnormalized', 'random' or 'symmetrized', got {} instead".format(laplacian))
        self.laplacian = laplacian
        self.opt_eps_jumps = opt_eps_jumps
        if self.eps is None and self.graph == 'eps':
            self.__optimum_epsilon()
    
    def __optimum_epsilon(self):
        """
        Compute epsilon
        
        To chose the minimum epsilon which guarantees G to be 
        connected, first, epsilon is set to be equal to the distance 
        from observation i = 0 to its nearest neighbour. Then
        we check if the Graph is connected, if it's not, epsilon
        is increased and the process is repeated until the Graph
        is connected
        """
        dist_matrix = pairwise_distances(self.X)
        self.eps = min(dist_matrix[0,1:])
        con = False
        while not con:
            self.eps = self.opt_eps_jumps * self.eps
            self.__construct_nearest_graph()
            con = self.cc == 1
            print('[INFO] Epsilon: {}'.format(self.eps))
        self.eps = np.round(self.eps, 3)
    
    def __heat_kernel(self, dist):
        """
        k(x, y) = exp(- ||x-y|| / sigma )
        """
        return np.exp(- (dist*dist)/self.sigma)
    
    def __rbf(self, dist):
        """
        k(x, y) = exp(- (1/2*sigma^2) * ||x-y||^2)
        """
        return np.exp(- dist**2/ (2* (self.sigma**2) ) )
    
    def __simple(self, *args):
        return 1
    
    def __construct_nearest_graph(self):
        """
        Compute weighted graph G
        """
        similarities_dic = {'heat kernel': self.__heat_kernel,
                            'simple':self.__simple,
                            'rbf':self.__rbf}
        
        dist_matrix = pairwise_distances(self.X)
        if self.graph == 'k-nearest':
            nn_matrix = np.argsort(dist_matrix, axis = 1)[:, 1 : self.k + 1]
        elif self.graph == 'eps':
            nn_matrix = np.array([ [index for index, d in enumerate(dist_matrix[i,:]) if d < self.eps and index != i] for i in range(self.n) ])
        # Weight matrix
        self._W = []
        for i in range(self.n):
            w_aux = np.zeros((1, self.n))
            similarities = np.array([ similarities_dic[self.weights](dist_matrix[i,v]) for v in nn_matrix[i]] )
            np.put(w_aux, nn_matrix[i], similarities)
            self._W.append(w_aux[0])
        self._W = np.array(self._W)
        # D matrix
        self._D = np.diag(self._W.sum(axis=1))
        # Check for connectivity
        self._G = self._W.copy() # Adjacency matrix
        self._G[self._G > 0] = 1
        G = nx.from_numpy_matrix(self._G)
        self.cc = nx.number_connected_components(G) # Multiplicity of lambda = 0
        if self.cc != 1:
            warnings.warn("Graph is not fully connected, Laplacian Eigenmaps may not work as expected")
            
    def __compute_unnormalized_laplacian(self):
        self.__construct_nearest_graph()
        self._L = self._D - self._W
        return self._L
    
    def __compute_normalized_random_laplacian(self):
        self.__construct_nearest_graph()
        self._Lr = np.eye(*self._W.shape) - (np.diag(1/self._D.diagonal())@self._W)
        return self._Lr
    
    def __compute_normalized_symmetrized_laplacian(self):
        self.__construct_nearest_graph()
        self.__compute_unnormalized_laplacian()
        d_tilde = np.diag(1/np.sqrt(self._D.diagonal()))
        self._Ls = d_tilde @ ( self._L @ d_tilde )
        return self._Ls
    
    def transform(self):
        """
        Compute embedding
        """
        
        m_options = {
            'unnormalized':self.__compute_unnormalized_laplacian,
            'random':self.__compute_normalized_random_laplacian,
            'symmetrized':self.__compute_normalized_symmetrized_laplacian
        }
        
        L = m_options[self.laplacian]()
        
        if self.laplacian == 'unnormalized':
            eigval, eigvec = eigh(L, self._D) # Generalized eigenvalue problem
        else:
            eigval, eigvec = np.linalg.eig(L)
            
        order = np.argsort(eigval)
        self.Y = eigvec[:, order[self.cc:self.cc+self.dim + 1]]
            
        return self.Y

In [3]:
import random
import torch
from torch_geometric import loader
from torch_geometric.data import Data

from sklearn.decomposition import PCA

cluster_data = loader.ClusterData(data, num_parts=1000, recursive=False)

iter_num = 1
with torch.no_grad():
    for cl in cluster_data:
        print("ISOMAP UPDATES: ITERATION " + str(iter_num))
        le = LE(X=cl.x, dim=100, k=51)
        cl.x = torch.from_numpy(le.transform())
        iter_num += 1

op_dict = {}
op_dict['embedding'] = cluster_data.data.x
op_dict['label'] = cluster_data.data.y
op_dict['train_idx'] = cluster_data.data.train_mask
op_dict['valid_idx'] = cluster_data.data.valid_mask
op_dict['test_idx'] = cluster_data.data.test_mask

torch.save(op_dict, '{}.pt'.format("EIGENMAP_EMBEDDINGS"))

Computing METIS partitioning...
Done!


ISOMAP UPDATES: ITERATION 1
ISOMAP UPDATES: ITERATION 2
ISOMAP UPDATES: ITERATION 3
ISOMAP UPDATES: ITERATION 4
ISOMAP UPDATES: ITERATION 5
ISOMAP UPDATES: ITERATION 6
ISOMAP UPDATES: ITERATION 7
ISOMAP UPDATES: ITERATION 8
ISOMAP UPDATES: ITERATION 9
ISOMAP UPDATES: ITERATION 10
ISOMAP UPDATES: ITERATION 11
ISOMAP UPDATES: ITERATION 12
ISOMAP UPDATES: ITERATION 13
ISOMAP UPDATES: ITERATION 14
ISOMAP UPDATES: ITERATION 15
ISOMAP UPDATES: ITERATION 16
ISOMAP UPDATES: ITERATION 17
ISOMAP UPDATES: ITERATION 18
ISOMAP UPDATES: ITERATION 19
ISOMAP UPDATES: ITERATION 20
ISOMAP UPDATES: ITERATION 21
ISOMAP UPDATES: ITERATION 22
ISOMAP UPDATES: ITERATION 23
ISOMAP UPDATES: ITERATION 24
ISOMAP UPDATES: ITERATION 25


/tmp/ipykernel_17197/4181453311.py:135: UserWarning: Graph is not fully connected, Laplacian Eigenmaps may not work as expected
  warnings.warn("Graph is not fully connected, Laplacian Eigenmaps may not work as expected")


ISOMAP UPDATES: ITERATION 26
ISOMAP UPDATES: ITERATION 27
ISOMAP UPDATES: ITERATION 28
ISOMAP UPDATES: ITERATION 29
ISOMAP UPDATES: ITERATION 30
ISOMAP UPDATES: ITERATION 31
ISOMAP UPDATES: ITERATION 32
ISOMAP UPDATES: ITERATION 33
ISOMAP UPDATES: ITERATION 34
ISOMAP UPDATES: ITERATION 35
ISOMAP UPDATES: ITERATION 36
ISOMAP UPDATES: ITERATION 37
ISOMAP UPDATES: ITERATION 38
ISOMAP UPDATES: ITERATION 39
ISOMAP UPDATES: ITERATION 40
ISOMAP UPDATES: ITERATION 41
ISOMAP UPDATES: ITERATION 42
ISOMAP UPDATES: ITERATION 43
ISOMAP UPDATES: ITERATION 44
ISOMAP UPDATES: ITERATION 45
ISOMAP UPDATES: ITERATION 46
ISOMAP UPDATES: ITERATION 47
ISOMAP UPDATES: ITERATION 48
ISOMAP UPDATES: ITERATION 49
ISOMAP UPDATES: ITERATION 50
ISOMAP UPDATES: ITERATION 51
ISOMAP UPDATES: ITERATION 52
ISOMAP UPDATES: ITERATION 53
ISOMAP UPDATES: ITERATION 54
ISOMAP UPDATES: ITERATION 55
ISOMAP UPDATES: ITERATION 56
ISOMAP UPDATES: ITERATION 57
ISOMAP UPDATES: ITERATION 58
ISOMAP UPDATES: ITERATION 59
ISOMAP UPDATES

/tmp/ipykernel_17197/4181453311.py:135: UserWarning: Graph is not fully connected, Laplacian Eigenmaps may not work as expected
  warnings.warn("Graph is not fully connected, Laplacian Eigenmaps may not work as expected")


...

/tmp/ipykernel_17197/4181453311.py:135: UserWarning: Graph is not fully connected, Laplacian Eigenmaps may not work as expected
  warnings.warn("Graph is not fully connected, Laplacian Eigenmaps may not work as expected")


ISOMAP UPDATES: ITERATION 791
ISOMAP UPDATES: ITERATION 792
ISOMAP UPDATES: ITERATION 793
ISOMAP UPDATES: ITERATION 794
ISOMAP UPDATES: ITERATION 795
ISOMAP UPDATES: ITERATION 796
ISOMAP UPDATES: ITERATION 797
ISOMAP UPDATES: ITERATION 798
ISOMAP UPDATES: ITERATION 799
ISOMAP UPDATES: ITERATION 800
ISOMAP UPDATES: ITERATION 801
ISOMAP UPDATES: ITERATION 802
ISOMAP UPDATES: ITERATION 803
ISOMAP UPDATES: ITERATION 804
ISOMAP UPDATES: ITERATION 805
ISOMAP UPDATES: ITERATION 806
ISOMAP UPDATES: ITERATION 807
ISOMAP UPDATES: ITERATION 808
ISOMAP UPDATES: ITERATION 809
ISOMAP UPDATES: ITERATION 810
ISOMAP UPDATES: ITERATION 811
ISOMAP UPDATES: ITERATION 812
ISOMAP UPDATES: ITERATION 813
ISOMAP UPDATES: ITERATION 814
ISOMAP UPDATES: ITERATION 815
ISOMAP UPDATES: ITERATION 816
ISOMAP UPDATES: ITERATION 817
ISOMAP UPDATES: ITERATION 818
ISOMAP UPDATES: ITERATION 819


/tmp/ipykernel_17197/4181453311.py:135: UserWarning: Graph is not fully connected, Laplacian Eigenmaps may not work as expected
  warnings.warn("Graph is not fully connected, Laplacian Eigenmaps may not work as expected")


ISOMAP UPDATES: ITERATION 820
ISOMAP UPDATES: ITERATION 821
ISOMAP UPDATES: ITERATION 822
ISOMAP UPDATES: ITERATION 823
ISOMAP UPDATES: ITERATION 824
ISOMAP UPDATES: ITERATION 825
ISOMAP UPDATES: ITERATION 826
ISOMAP UPDATES: ITERATION 827
ISOMAP UPDATES: ITERATION 828
ISOMAP UPDATES: ITERATION 829
ISOMAP UPDATES: ITERATION 830
ISOMAP UPDATES: ITERATION 831
ISOMAP UPDATES: ITERATION 832
ISOMAP UPDATES: ITERATION 833
ISOMAP UPDATES: ITERATION 834
ISOMAP UPDATES: ITERATION 835
ISOMAP UPDATES: ITERATION 836
ISOMAP UPDATES: ITERATION 837
ISOMAP UPDATES: ITERATION 838
ISOMAP UPDATES: ITERATION 839
ISOMAP UPDATES: ITERATION 840
ISOMAP UPDATES: ITERATION 841
ISOMAP UPDATES: ITERATION 842
ISOMAP UPDATES: ITERATION 843
ISOMAP UPDATES: ITERATION 844
ISOMAP UPDATES: ITERATION 845
ISOMAP UPDATES: ITERATION 846
ISOMAP UPDATES: ITERATION 847
ISOMAP UPDATES: ITERATION 848
ISOMAP UPDATES: ITERATION 849
ISOMAP UPDATES: ITERATION 850
ISOMAP UPDATES: ITERATION 851
ISOMAP UPDATES: ITERATION 852
ISOMAP UPD

RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 640 vs 582

In [5]:
torch.save(op_dict, '{}.pt'.format("EIGENMAP_EMBEDDINGS"))

RuntimeError: [enforce fail at inline_container.cc:300] . unexpected pos 640 vs 582

In [17]:
!sudo growpart /dev/xvda1 1

WARN: unknown label 
failed [sfd_dump:1] sfdisk --unit=S --dump /dev/xvda1
sfdisk: /dev/xvda1: does not contain a recognized partition table
FAILED: failed to dump sfdisk info for /dev/xvda1
